# profile and pstats - Performance Analysis

Purpose:	Performance analysis of Python programs.

The profile module provides APIs for collecting and analyzing statistics about how Python source consumes processor resources.

##### Note
This output reports in this section have been reformatted to fit on the page. Lines ending with backslash (\) are continued on the next line.

## Running the Profiler

The most basic starting point in the profile module is run(). It takes a string statement as argument, and creates a report of the time spent executing different lines of code while running the statement.

In [1]:
# profile_fibonacci_raw.py

import profile

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

profile.run('print(fib_seq(20));print()')

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]

         57402 function calls (112 primitive calls) in 0.205 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(acquire)
       21    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.205    0.205 :0(exec)
       20    0.000    0.000    0.000    0.000 :0(extend)
        3    0.000    0.000    0.000    0.000 :0(getpid)
        3    0.000    0.000    0.000    0.000 :0(isinstance)
        2    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.000    0.000    0.000    0.000 :0(urandom)
     21/1    0.000    0.000    0.205    0.205 <ipython-input-1-a9266880fffc>:13(fib_seq)
 57291/21    0.204    0.000    0.204    0.010 <ipython-input-1-a9266880fffc>:5(fib)
        1    0.000    0.000    0.205    0.2

The details about where time was spent are broken out by function in the listing showing the number of calls, total time spent in the function, time per call (tottime/ncalls), cumulative time spent in a function, and the ratio of cumulative time to primitive calls.

Not surprisingly, most of the time here is spent calling fib() repeatedly. Adding a cache decorator reduces the number of recursive calls, and has a big impact on the performance of this function.

In [2]:
# profile_fibonacci_memoized.py
import functools
import profile


@functools.lru_cache(maxsize=None)
def fib(n):
    # from literateprograms.org
    # http://bit.ly/hlOQ5m
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n - 1))
    seq.append(fib(n))
    return seq


if __name__ == '__main__':
    profile.run('print(fib_seq(20)); print()')

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]

         132 function calls (112 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(acquire)
       21    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.001    0.001 :0(exec)
       20    0.000    0.000    0.000    0.000 :0(extend)
        3    0.000    0.000    0.000    0.000 :0(getpid)
        3    0.000    0.000    0.000    0.000 :0(isinstance)
        2    0.000    0.000    0.001    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.000    0.000    0.000    0.000 :0(urandom)
     21/1    0.000    0.000    0.001    0.001 <ipython-input-2-6d87e85147c0>:18(fib_seq)
       21    0.000    0.000    0.000    0.000 <ipython-input-2-6d87e85147c0>:6(fib)
        1    0.000    0.000    0.001    0.001

By remembering the Fibonacci value at each level, most of the recursion is avoided. The ncalls count for fib() shows that it never recurses.

## Running in a Context

Sometimes, instead of constructing a complex expression for run(), it is easier to build a simple expression and pass it parameters through a context, using runctx().

In this example, the value of n is passed through the local variable context instead of being embedded directly in the statement passed to runctx().

In [3]:
# profile_runctx.py

import profile
import functools

@functools.lru_cache(maxsize=None)
def fib(n):
    # from literateprograms.org
    # http://bit.ly/hlOQ5m
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n - 1))
    seq.append(fib(n))
    return seq

if __name__ == '__main__':
    profile.runctx(
        'print(fib_seq(n));print()', globals(), {'n':20})

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]

         132 function calls (112 primitive calls) in 0.003 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(acquire)
       21    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.003    0.003 :0(exec)
       20    0.000    0.000    0.000    0.000 :0(extend)
        3    0.000    0.000    0.000    0.000 :0(getpid)
        3    0.000    0.000    0.000    0.000 :0(isinstance)
        2    0.000    0.000    0.003    0.001 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.001    0.000    0.001    0.000 :0(urandom)
     21/1    0.000    0.000    0.000    0.000 <ipython-input-3-b0ff3ede32d0>:18(fib_seq)
       21    0.000    0.000    0.000    0.000 <ipython-input-3-b0ff3ede32d0>:6(fib)
        1    0.000    0.000    0.003    0.003

## pstats: Saving and Working With Statistics

The standard report created by the profile functions is not very flexible. However, custom reports can be produced by saving the raw profiling data from run() and runctx() and processing it separately with the pstats.Stats class.

This example runs several iterations of the same test and combines the results.

The output report is sorted in descending order of cumulative time spent in the function and the directory names are removed from the printed filenames to conserve horizontal space on the page.

In [4]:
# profile_stats.py

import cProfile as profile
import pstats
import functools

@functools.lru_cache(maxsize=None)
def fib(n):
    # from literateprograms.org
    # http://bit.ly/hlOQ5m
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n - 1))
    seq.append(fib(n))
    return seq

# create 5 sets of stats
for i in range(5):
    filename = 'profile_stats_{}.stats'.format(i)
    profile.run('print({}, fib_seq(20))'.format(i), filename)

# read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_{}.stats'.format(i))

# clean up filenames for the report
stats.strip_dirs()

# sort the stats by the cumulative time spent
stats.sort_stats('cumulative')

stats.print_stats()

0 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
1 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
2 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
3 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
4 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
Thu Apr  6 23:26:31 2017    profile_stats_0.stats
Thu Apr  6 23:26:31 2017    profile_stats_1.stats
Thu Apr  6 23:26:31 2017    profile_stats_2.stats
Thu Apr  6 23:26:31 2017    profile_stats_3.stats
Thu Apr  6 23:26:31 2017    profile_stats_4.stats

         598 function calls (498 primitive calls) in 0.003 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.003    0.001 {built-in method builtins.exec}
        5    0.000    0.000    0.0

## Limiting Report Contents

The output can be restricted by function. This version only shows information about the performance of fib() and fib_seq() by using a regular expression to match the desired filename:lineno(function) values.

The regular expression includes a literal left parenthesis (() to match against the function name portion of the location value.

In [5]:
# profile_stats_restricted.py

import profile
import pstats
import functools

@functools.lru_cache(maxsize=None)
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib (n-2)
    
def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.extend(fib(n))
    return seq

# Read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_{}.stats'.format(i))
stats.strip_dirs()
stats.sort_stats('cumulative')

# limit output to lines with "(fib" in them
stats.print_stats('\(fib')

Thu Apr  6 23:26:31 2017    profile_stats_0.stats
Thu Apr  6 23:26:31 2017    profile_stats_1.stats
Thu Apr  6 23:26:31 2017    profile_stats_2.stats
Thu Apr  6 23:26:31 2017    profile_stats_3.stats
Thu Apr  6 23:26:31 2017    profile_stats_4.stats

         598 function calls (498 primitive calls) in 0.003 seconds

   Ordered by: cumulative time
   List reduced from 20 to 2 due to restriction <'\\(fib'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    105/5    0.000    0.000    0.000    0.000 <ipython-input-4-ebcae057bc8e>:19(fib_seq)
       21    0.000    0.000    0.000    0.000 <ipython-input-4-ebcae057bc8e>:7(fib)




## Caller / Callee Graphs

Stats also includes methods for printing the callers and callees of functions.

The arguments to print_callers() and print_callees() work the same as the restriction arguments to print_stats(). The output shows the caller, callee, number of calls, and cumulative time.

In [6]:
# profile_stats_callers.py
import cProfile as profile
import pstats
import functools

@functools.lru_cache(maxsize=None)
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib (n-2)
    
def fib_seq(n):
    seq = []
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.extend(fib(n))
    return seq

# Read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_{}.stats'.format(i))
stats.strip_dirs()
stats.sort_stats('cumulative')

print('INCOMING CALLERS:')
stats.print_callers('\(fib')

print('OUTGOING CALLEES:')
stats.print_callees('\(fib')

INCOMING CALLERS:
   Ordered by: cumulative time
   List reduced from 20 to 2 due to restriction <'\\(fib'>

Function                                    was called by...
                                                ncalls  tottime  cumtime
<ipython-input-4-ebcae057bc8e>:19(fib_seq)  <-   100/5    0.000    0.000  <ipython-input-4-ebcae057bc8e>:19(fib_seq)
                                                     5    0.000    0.000  <string>:1(<module>)
<ipython-input-4-ebcae057bc8e>:7(fib)       <-      21    0.000    0.000  <ipython-input-4-ebcae057bc8e>:19(fib_seq)


OUTGOING CALLEES:
   Ordered by: cumulative time
   List reduced from 20 to 2 due to restriction <'\\(fib'>

Function                                    called...
                                                ncalls  tottime  cumtime
<ipython-input-4-ebcae057bc8e>:19(fib_seq)  ->      21    0.000    0.000  <ipython-input-4-ebcae057bc8e>:7(fib)
                                                 100/5    0.000    0.000  <ipy

In [7]:
! rm profile_stats_[0-4].stats